In [7]:
#Solution adapted from Kris C Naik
#GitHub https://github.com/krishnaik06/Stock-Price-Prediction-using-Keras-and-Recurrent-Neural-Networ/blob/master/rnn.py
import matplotlib.pyplot as plt
import pandas as pd
import statistics
import datetime as dt
import urllib.request, json
import pickle
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import autocorrelation_plot
#from app.model.preprocessor import Preprocessor as img_prep #image preprocessing

In [8]:
import pandas_datareader as dr
from datetime import datetime
import pandas_datareader.data as web

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D

from keras.optimizers import SGD
from keras.utils import np_utils
from scipy import misc
import glob


In [9]:
start=datetime(2015,1,1)
end=datetime(2019,4,1)

In [10]:
# tesla = web.DataReader('TSLA', 'google', start,end)
microsoft = dr.data.get_data_yahoo('MSFT', start=start, end=end)

dataset = microsoft
close_data = microsoft['Close']
dates = pd.DataFrame(close_data.index)

dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,47.419998,46.540001,46.660000,46.759998,27913900.0,42.418739
2015-01-05,46.730000,46.250000,46.369999,46.330002,39673900.0,42.028660
2015-01-06,46.750000,45.540001,46.380001,45.650002,36447900.0,41.411785
2015-01-07,46.459999,45.490002,45.980000,46.230000,29114100.0,41.937946
2015-01-08,47.750000,46.720001,46.750000,47.590000,29645200.0,43.171677


In [12]:
model = Sequential()

model.add(Convolution2D(filters=8,kernel_size=100,strides=2, kernel_initializer='VarianceScaling', activation='relu', 
                        input_shape=(len(dates),1), data_format='channels_first'))
model.add(MaxPooling2D(pool_size=500,strides=2))

model.add(Convolution2D(filters=16,kernel_size=5,strides=1, kernel_initializer='VarianceScaling', activation='relu'))
model.add(MaxPooling2D(pool_size=100,strides=1))

model.add(Conv2D(16,5,1, kernel_initializer='VarianceScaling', activation='relu'))
model.add(MaxPooling2D(pool_size=100 , strides=1))

model.add(Dense(units=10, kernel_initializer='VarianceScaling',activation='softmax'))
#model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", optimizer='sgd', metrics=['accuracy'])


print(close_data.reshape([1,1069,1])[0,0:6,0])
model.fit(dates['Date'].reshape([1,1069,1]), close_data.reshape([1,1069,1]), batch_size=3, epochs=30)

ValueError: Input 0 of layer conv2d_3 is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [None, 1069, 1]

In [ ]:
class LiteCNN:
    def __init__(self):
        sel.layers=[]
        #size of pooling area for max pooling
        self.pool_size=None

    def load_weight(self, weights):
        assert not self.layers, "Weights can only be loaded once!"
        #add the saved matrix values to the convolutional network
        for k in range(len(weights.keys())):
            self.layers.append(weights['layer{}'.format(k)])
    def predict(self, X):
        #here is where the network magic happens at a high level
        h=self.cnn_layer(Xm, layer_i=0,border_mode='full'); X=h
        h = self.relu_layer(X); X=h;
        h = self.cnn_layer(X, layer_i=2, border_mode='valid'); X=h
        h = self.relu_layer(X); X=h;
        h = self.maxpooling_layer(X); x=h;
        h = self.dropout_layer(X, .25); x=h;
        h = self.flatten_layer(X, layer_i=7); X = h;
        h = self.dense_layer(X, fully, layer_i=0); X = h;
        h = self.softmax_layer2D(X); x = h;
        max_i = self.classify(X)
        return max_i[0]
    
#given our feature map we've learned from convolving around the image
    #lets make it more dense by performing pooling, specifically max pooling
    #we'll select the max values from the image matrix and use that as our new feature map
    def maxpooling_layer(self, convolved_features):
        #given our learned features and images
        nb_features = convolved_features.shape[0]
        nb_images = convolved_features.shape[1]
        conv_dim = convolved_features.shape[2]
        res_dim = int(conv_dim / self.pool_size)       #assumed square shape

        #initialize our more dense feature list as empty
        pooled_features = np.zeros((nb_features, nb_images, res_dim, res_dim))
        #for each image
        for image_i in range(nb_images):
            #and each feature map
            for feature_i in range(nb_features):
                #begin by the row
                for pool_row in range(res_dim):
                    #define start and end points
                    row_start = pool_row * self.pool_size
                    row_end   = row_start + self.pool_size

                    #for each column (so its a 2D iteration)
                    for pool_col in range(res_dim):
                        #define start and end points
                        col_start = pool_col * self.pool_size
                        col_end   = col_start + self.pool_size

                        #define a patch given our defined starting ending points
                        patch = convolved_features[feature_i, image_i, row_start : row_end,col_start : col_end]
                        #then take the max value from that patch
                        #store it. this is our new learned feature/filter
                        pooled_features[feature_i, image_i, pool_row, pool_col] = np.max(patch)
        return pooled_features

    #convolution is the most important of the matrix operations here
    #well define our input, lauyer number, and a border mode (explained below)
    def cnn_layer(self, X, layer_i=0, border_mode = "full"):
        #we'll store our feature maps and bias value in these 2 vars
        features = self.layers[layer_i]["param_0"]
        bias = self.layers[layer_i]["param_1"]
        #how big is our filter/patch?
        patch_dim = features[0].shape[-1]
        #how many features do we have?
        nb_features = features.shape[0]
        #How big is our image?
        image_dim = X.shape[2] #assume image square
        #R G B values
        image_channels = X.shape[1]
        #how many images do we have?
        nb_images = X.shape[0]

        #With border mode "full" you get an output that is the "full" size as the input. 
        #That means that the filter has to go outside the bounds of the input by "filter size / 2" - 
        #the area outside of the input is normally padded with zeros.
        if border_mode == "full":
            conv_dim = image_dim + patch_dim - 1
        #With border mode "valid" you get an output that is smaller than the input because 
        #the convolution is only computed where the input and the filter fully overlap.
        elif border_mode == "valid":
            conv_dim = image_dim - patch_dim + 1

        #we'll initialize our feature matrix
        convolved_features = np.zeros((nb_images, nb_features, conv_dim, conv_dim));
        #then we'll iterate through each image that we have
        for image_i in range(nb_images):
            #for each feature 
            for feature_i in range(nb_features):
                #lets initialize a convolved image as empty
                convolved_image = np.zeros((conv_dim, conv_dim))
                #then for each channel (r g b )
                for channel in range(image_channels):
                    #lets extract a feature from our feature map
                    feature = features[feature_i, channel, :, :]
                    #then define a channel specific part of our image
                    image   = X[image_i, channel, :, :]
                    #perform convolution on our image, using a given feature filter
                    convolved_image += self.convolve2d(image, feature, border_mode);

                #add a bias to our convoved image
                convolved_image = convolved_image + bias[feature_i]
                #add it to our list of convolved features (learnings)
                convolved_features[image_i, feature_i, :, :] = convolved_image
        return convolved_features

    #In a dense layer, every node in the layer is connected to every node in the preceding layer.
    def dense_layer(self, X, layer_i=0):
        #so we'll initialize our weight and bias for this layer
        W = self.layers[layer_i]["param_0"]
        b = self.layers[layer_i]["param_1"]
        #and multiply it by our input (dot product)
        output = np.dot(X, W) + b
        return output

    @staticmethod

    #so what does the convolution operation look like?, given an image and a feature map (filter)
    def convolve2d(image, feature, border_mode="full"):
        #we'll define the tensor dimensions of the image and the feature
        image_dim = np.array(image.shape)
        feature_dim = np.array(feature.shape)
        #as well as a target dimension
        target_dim = image_dim + feature_dim - 1
        #then we'll perform a fast fourier transform on both the input and the filter
        #performing a convolution can be written as a for loop but for many convolutions
        #this approach is too comp. expensive/slow. it can be performed orders of magnitude
        #faster using a fast fourier transform. 
        fft_result = np.fft.fft2(image, target_dim) * np.fft.fft2(feature, target_dim)
        #and set the result to our target 
        target = np.fft.ifft2(fft_result).real

        if border_mode == "valid":
            # To compute a valid shape, either np.all(x_shape >= y_shape) or
            # np.all(y_shape >= x_shape).
            #decide a target dimension to convolve around
            valid_dim = image_dim - feature_dim + 1
            if np.any(valid_dim < 1):
                valid_dim = feature_dim - image_dim + 1
            start_i = (target_dim - valid_dim) // 2
            end_i = start_i + valid_dim
            target = target[start_i[0]:end_i[0], start_i[1]:end_i[1]]
        return target

    def relu_layer(x):
        #turn all negative values in a matrix into zeros
        z = np.zeros_like(x)
        return np.where(x>z,x,z)

    def softmax_layer2D(w):
        #this function will calculate the probabilities of each
        #target class over all possible target classes. 
        maxes = np.amax(w, axis=1)
        maxes = maxes.reshape(maxes.shape[0], 1)
        e = np.exp(w - maxes)
        dist = e / np.sum(e, axis=1, keepdims=True)
        return dist

    #affect the probability a node will be turned off by multiplying it
    #by a p values (.25 we define)
    def dropout_layer(X, p):
        retain_prob = 1. - p
        X *= retain_prob
        return X

    #get the largest probabililty value from the list
    def classify(X):
        return X.argmax(axis=-1)

    #tensor transformation, less dimensions
    def flatten_layer(X):
        flatX = np.zeros((X.shape[0],np.prod(X.shape[1:])))
        for i in range(X.shape[0]):
            flatX[i,:] = X[i].flatten(order='C')
        return flatX

In [ ]:
 # First calculate the mid prices from the highest and lowest

train_data = model_dataset[:750]
test_data = model_dataset[750:]

print(train_data.transpose())
print(len(test_data))

In [13]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_data)

NameError: name 'train_data' is not defined

In [ ]:
# Creating a data structure with 63 (approx. 1 quarter) timesteps and 1 output (next day value)
X_train = []
y_train = []
for i in range(63, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-63:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(X_train.shape[1])
print(len(y_train))

In [ ]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
#https://keras.io/layer/recurrent
from keras.models import Sequential
from keras.layers import Dense #output layer
from keras.layers import LSTM
from keras.layers import Dropout #for each Epox deactivate those that result in 0

In [ ]:
# Initialising the RNN
regressor = Sequential()

In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

Model, fit and plot data using regressor calculated above.

In [ ]:
# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 30, batch_size = 100)

In [ ]:
# Setting Real stock price
real_stock_price = test_data

# Getting the predicted stock price
dataset_total = dataset['Close']
inputs = dataset_total[len(dataset_total) - len(test_data) - 63:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(63, 380):
    X_test.append(inputs[i-63:i, 0])
X_test = np.array(X_test)


X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results

plt.plot(real_stock_price, color = 'red', label = 'Real Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
from pandas.tseries.offsets import DateOffset

In [ ]:
future_dates = [dataset.index[-1] + DateOffset(days=x) for x in range(1,365)]

In [ ]:
print(regressor.summary())

In [ ]:
x = np.array(future_dates.index)

future_dates['forecast'] = regressor.predict(np.array(future_dates.index))